In [5]:
import numpy as np
import xarray as xr
import geopandas as gpd
from shapely.geometry import Point
import cartopy.crs as ccrs

base_path = '/home/jez/Bias_Correction/'
observations_path = f'{base_path}data/ProcessedData/NST_Observations.nc'
region_shapefile_path = f'{base_path}data/Ross_Region_Shapefile/ross_region.shp'
antarctica_shapefile_path = f'{base_path}data/Antarctica_Shapefile/antarctica_shapefile.shp'
out_path = f'{base_path}data/ProcessedData/NST_Observations_Subset.nc'

ds = xr.open_dataset(observations_path)
region_gdf = gpd.read_file(region_shapefile_path)
antarctica_gdf = gpd.read_file(antarctica_shapefile_path)
map_proj = ccrs.Orthographic(central_longitude=0.0, central_latitude=-90, globe=None)


In [6]:
ds

<xarray.Dataset>
Dimensions:        (Station_Lower: 216, Year: 42, Month: 12, Day: 31)
Coordinates:
  * Station_Lower  (Station_Lower) object 'a028' 'a028-a' ... 'zoraida'
  * Year           (Year) float64 1.98e+03 1.981e+03 ... 2.02e+03 2.021e+03
  * Month          (Month) float64 1.0 2.0 3.0 4.0 5.0 ... 9.0 10.0 11.0 12.0
  * Day            (Day) float64 1.0 2.0 3.0 4.0 5.0 ... 28.0 29.0 30.0 31.0
    Lat(°C)        (Station_Lower) float64 ...
    Lon(°C)        (Station_Lower) float64 ...
    Elevation(m)   (Station_Lower) float64 ...
    Institution    (Station_Lower) object ...
Data variables:
    Temperature()  (Station_Lower, Year, Month, Day) float64 ...

In [7]:
points = [Point(lon,lat) for lon,lat in zip(ds['Lon(°C)'],ds['Lat(°C)'])]
points_gdf = gpd.GeoDataFrame(crs='epsg:4326', geometry=points)

In [9]:
mask = [region_gdf.to_crs(map_proj).contains(points_gdf[i:i+1].reset_index().to_crs(map_proj)) for i in range(points_gdf.shape[0])]
mask = np.array(mask).reshape(-1)

In [10]:
subset_ds = ds.sel(Station_Lower=mask)

In [11]:
subset_ds

<xarray.Dataset>
Dimensions:        (Station_Lower: 57, Year: 42, Month: 12, Day: 31)
Coordinates:
  * Station_Lower  (Station_Lower) object 'alexander-tt' ... 'windless-bight'
  * Year           (Year) float64 1.98e+03 1.981e+03 ... 2.02e+03 2.021e+03
  * Month          (Month) float64 1.0 2.0 3.0 4.0 5.0 ... 9.0 10.0 11.0 12.0
  * Day            (Day) float64 1.0 2.0 3.0 4.0 5.0 ... 28.0 29.0 30.0 31.0
    Lat(°C)        (Station_Lower) float64 -79.0 -76.72 -77.6 ... -77.87 -77.73
    Lon(°C)        (Station_Lower) float64 170.8 163.0 161.1 ... 166.9 167.7
    Elevation(m)   (Station_Lower) float64 55.0 150.0 1.75e+03 ... 9.0 40.0
    Institution    (Station_Lower) object 'AMRC' 'PNRA' 'AMRC' ... 'AMRC' 'AMRC'
Data variables:
    Temperature()  (Station_Lower, Year, Month, Day) float64 ...

In [12]:
import xarray as xr
import pandas as pd
import datetime
from src.helper_functions import calculate_time

pd.set_option('display.max_rows', 500)

In [20]:
def stack_year_month_day(ds):
    ds_stacked = ds.stack(time=("Year", "Month", "Day"))
    ds_stacked = ds_stacked.dropna('time', how='all')

    time_coord = []
    for i in ds_stacked.time.data:
        y,m,d = i
        calctime = datetime.datetime(year=int(y), month=int(m), day=int(d))
        time_coord.append(calctime)

    ds_stacked = ds_stacked.assign_coords({'newtime':("time", time_coord)})
    ds_stacked = ds_stacked.swap_dims({'time': 'newtime'})
    ds_stacked = ds_stacked.drop('time')
    ds_stacked = ds_stacked.rename({'newtime':'time'})

    return(ds)

In [21]:
ds_subset_stacked = stack_year_month_day(subset_ds)

In [19]:
ds_subset_stacked

In [22]:
df = ds_subset_stacked.to_dataframe()
df = df.dropna()
df_summary = df.groupby(['Station_Lower','Year']).describe().T
df_summary.loc[(('Temperature()'), ('count','mean','min','max')), :].T.head(500)

Temperature()                                 
                               count       mean        min        max
Station_Lower   Year                                                 
alexander-tt    2011.0         332.0 -22.910402 -51.780000  -1.787500
                2013.0          33.0  -7.059109 -16.250000  -3.816667
arelis          1990.0         344.0 -17.186120 -35.700000   1.987500
                1991.0         365.0 -15.219429 -33.712500   1.750000
                1992.0         366.0 -16.191950 -38.075000   0.775000
                1993.0         365.0 -17.000644 -34.650000   1.625000
                1994.0         364.0 -17.854910 -36.325000  -0.037500
                1995.0         365.0 -17.232317 -36.550000   3.525000
                1996.0         366.0 -15.117446 -33.037500   1.687500
                1997.0         365.0 -16.548657 -35.475000   1.100000
                1998.0         365.0 -16.560818 -34.037500   1.287500
                1999.0         365.0 -15.807931 -35.200000   0.150000
                2000.0         365.0 -16.111431 -33.137500  -2.012500
                2001.0         364.0 -16.704836 -37.920000   4.260000
                2002.0         363.0 -16.501483 -35.800000   6.550000
                2003.0         365.0 -15.973446 -33.775000   1.925000
                2004.0         366.0 -17.588088 -39.337500   1.637500
                2005.0         364.0 -16.241391 -39.200000   1.975000
                2006.0         365.0 -16.028535 -35.100000   1.983333
                2007.0         364.0 -14.057801 -30.525000   3.587500
                2008.0         366.0 -15.227645 -36.875000   2.987500
                2009.0         365.0 -15.880749 -34.375000   0.300000
                2010.0         365.0 -15.921894 -37.437500   2.971429
                2011.0         365.0 -14.675537 -33.475000   1.825000
                2012.0         366.0 -15.356437 -32.262500   1.512500
                2013.0         364.0 -15.570344 -34.037500   1.112500
                2014.0         365.0 -15.119341 -34.775000   1.475000
                2015.0         365.0 -16.374531 -34.625000   3.442857
                2016.0         366.0 -16.895327 -36.562500   1.471429
                2017.0         365.0 -15.240303 -31.700000   0.300000
                2018.0         365.0 -14.872359 -33.975000   2.262500
                2019.0         356.0 -14.577350 -34.600000   0.137500
                2020.0         366.0 -15.228241 -33.762500   1.325000
                2021.0           1.0  -3.875000  -3.875000  -3.875000
asgard          1980.0         331.0 -23.025096 -39.185714  -9.275000
                1981.0         349.0 -22.703206 -41.080000  -6.812500
                1982.0         364.0 -23.301758 -42.375000  -3.687500
bowers          1986.0          89.0 -27.721758 -51.575000 -13.433333
brianna         1994.0          32.0  -6.701544 -10.837500  -3.600000
                1995.0         364.0 -22.819575 -53.925000  -2.987500
                1996.0         334.0 -21.318465 -46.512500  -2.912500
                1997.0         156.0 -21.514317 -44.937500  -3.937500
                2000.0         348.0 -21.717189 -49.862500  -6.025000
                2001.0         298.0 -24.551279 -48.000000  -6.150000
                2002.0         351.0 -22.332754 -48.587500  -4.750000
                2003.0          59.0 -11.119199 -23.700000  -1.562500
                2008.0          98.0 -14.538929 -28.583333  -5.200000
                2009.0         365.0 -22.802335 -55.287500  -3.262500
                2010.0         365.0 -22.793146 -48.542857  -3.512500
                2011.0         364.0 -20.255181 -54.716667  -1.650000
                2013.0         365.0 -22.468985 -47.537500  -1.362500
                2014.0         365.0 -21.146295 -47.285714  -3.787500
                2015.0         341.0 -23.845906 -49.300000  -1.300000
cape-bird       1999.0         338.0 -15.264167 -36.185714  -0.550000
                2000.0         36

In [ ]:
points = [Point(lon,lat) for lon,lat in zip(ds['Lon(°C)'],ds['Lat(°C)'])]
points_gdf = gpd.GeoDataFrame(crs='epsg:4326', geometry=points)

mask = [region_gdf.to_crs(map_proj).contains(points_gdf[i:i+1].reset_index().to_crs(map_proj)) for i in range(points_gdf.shape[0])]
mask = np.array(mask).reshape(-1)

subset_ds = ds.sel(Station_Lower=mask)

subset_ds['Institution'] = subset_ds.Institution.astype(str)
subset_ds.to_netcdf(out_path)
